In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
OPENAI_KEY1 = "sk-XXX"
OPENAI_KEY2 = "sk-XX"

## Use OpenAI

Set you `OPENAI_API_KEY` environment variable.

In [2]:
from manifest import Manifest
from manifest.connections.client_pool import ClientConnection

openai_ada = ClientConnection(
    client_name="openai",
    client_connection=OPENAI_KEY1,
    engine="text-ada-001"
)

openai_curie = ClientConnection(
    client_name="openai",
    client_connection=OPENAI_KEY2,
    engine="text-curie-001"
)

manifest = Manifest(client_pool=[openai_ada, openai_curie], client_pool_schedule="round_robin")

In [3]:
res = manifest.run("What model are you?", temperature=0.0)
print(manifest.client_pool.current_client_id)
print(res)
res = manifest.run("What model are you?", temperature=0.0)
print(manifest.client_pool.current_client_id)
print(res)

0
I am a model.
1
I am a MacBook Pro with a retina


## With Async

In [4]:
import nest_asyncio
# This is required for asyncio.run(...) to work in Jupyter notebooks.
nest_asyncio.apply()

In [5]:
from manifest import Manifest
from manifest.connections.client_pool import ClientConnection

openai_ada = ClientConnection(
    client_name="openai",
    client_connection=OPENAI_KEY1,
    engine="text-ada-001"
)

openai_babbage = ClientConnection(
    client_name="openai",
    client_connection=OPENAI_KEY2,
    engine="text-babbage-001"
)

openai_curie = ClientConnection(
    client_name="openai",
    client_connection=OPENAI_KEY2,
    engine="text-curie-001"
)

manifest = Manifest(client_pool=[openai_ada, openai_babbage, openai_curie], client_pool_schedule="round_robin")
manifest_single_client = Manifest(client_pool=[openai_babbage])

In [6]:
import time
import asyncio

prompts = [f"Tell me something interesting about {i}" for i in range(400)]
st = time.time()
for pmt in prompts:
    _ = manifest_single_client.run(pmt, max_tokens=30)
print(f"For loop: {time.time() - st :.2f}")

print("Running with async single client")
st = time.time()
_ = asyncio.run(manifest_single_client.arun_batch(prompts, max_tokens=30, chunk_size=-1))
print(f"Async loop: {time.time() - st :.2f}")

print("Running with async two clients but not chunking")
st = time.time()
_ = asyncio.run(manifest.arun_batch(prompts, max_tokens=30, chunk_size=-1))
print(f"Async loop: {time.time() - st :.2f}")

print("Running with async two clients and chunk size")
st = time.time()
_ = asyncio.run(manifest.arun_batch(prompts, max_tokens=30, chunk_size=20))
print(f"Async loop: {time.time() - st :.2f}")

For loop: 128.68
Running with async single client
Running 1 tasks across all clients.
Async loop: 4.02
Running with async two clients but not chunking
Running 1 tasks across all clients.
Async loop: 3.92
Running with async two clients and chunk size
Running 20 tasks across all clients.
Async loop: 1.44
